# Ling 450/807 SFU - Assignment 1

Virginia Uhi, Eunice Wong, & Han Zhang


## Import packages

We import everything we will need here at the beginning and load the spaCy language model. Note that we are using the small English model. One thing you could try is to download and load [other models for English](https://spacy.io/models/en) and compare the results. 

In [24]:
# Importing Spacy 
import spacy
import re
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)


## Approach 1: Using regular expressions

In [25]:
# The starter file loads and processes only one file at a time. We are using the command to do 5 files at the same time

# First we take five text files out 

five_files = ["A1_data/5c1548a31e67d78e2771624f.txt", "A1_data/5c489df91e67d78e271d66c5.txt", "A1_data/5c182ac21e67d78e277944ad.txt", "A1_data/5c28972a795bd2fac69fa974.txt", "A1_data/5c29beda1e67d78e27b74939.txt"]

# Creat a file
file_path = "Regular_Expression_All.txt"
with open(file_path, 'w', encoding='utf-8'):
    pass

# Process five text files at the same time, and have all the quotes in the five files can be extracted and put in the same file
with open ("Regular_Expression_All.txt", 'w', encoding = "utf-8") as output:
    def get_quotes(text):
        quotes = re.findall(r'["“](.*?)[”"]', text) 
            # so that every time as soon as Spacy identify any quotation marks (either straight or curly), the quoted content with the quotation mark will be extracted
        return(quotes)
    
    for file_path in five_files: 
        output.write(file_path + "\n")
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()
                # The starter code seperate the text into sentences at the begining; However, for the sake of better identifying the sentence, we decide to not seperate the sentences first

            found_quotes = get_quotes(text) 
            print(found_quotes)
            for quote in found_quotes:
                output.write(str(quote) + "\n")  # Using the "\n" to make the quotes listed line by line

    

['Some of those ‘soft’ skills are in short supply, but they’re what employers are looking for,', 'There’s a societal cost to having someone remain unemployed,', 'There are health-care costs. There are welfare costs. There can be criminal justice costs,', 'road map', 'should move to skills-based hiring,', 'practices that prioritize … credentials and experience.', 'These are things that are the least susceptible to technological disruption,']
["It's good that we take the time early on to completely shut hockey off and just relax and then the closer we get to coming back, we've got to get the minds ready,", "Most guys were here last year and we learned a lesson from last year when we weren't as good after the break. We know what to do this time and what not to.", 'He enabled us to at least hang in there and get a point out of this game,', 'The positive is you came in here against one of the best teams and in my opinion, we could have had that game.', "I don't think we played our best game

## Approach 2: Using spaCy's Matcher

This approach is based on notebooks by [William J.B. Mattingly](https://wjbmattingly.com/). His book, Introduction to Python for Humanists, is available online from the [SFU Library](https://sfu-primo.hosted.exlibrisgroup.com/permalink/f/usv8m3/01SFUL_ALMA51476999620003611). 

For more on spaCy's Matcher, see Advanced NLP with spaCy, [chapter 2](https://course.spacy.io/en/chapter2)). 

We have already loaded everything we need at the beginning of this notebook (imported Matcher, assigned it to a `matcher` object), so now we can use it. 

## Finding quotes and speakers

In [5]:
# convert it to a spacy doc
doc = nlp(text)

### Finding proper nouns

In [ ]:
# This is optional. It just tells you who are the people mentioned. You can use it later if you want to find out the speakers of the quotes

# matcher = Matcher(nlp.vocab)
# pattern_n = [{"POS": "PROPN"}]
# matcher.add("PROPER_NOUNS", [pattern_n], greedy="LONGEST")
# doc = nlp(text)
# matches = matcher(doc)
# print (len(matches))
# for match in matches[:10]:
    #print (match, doc[match[1]:match[2]])
    
## You can try to extract full names by adding multi-word nouns, http://spacy.pythonhumanities.com/02_02_matcher.html

In [6]:
# We skip the he code above and try another method
# the starter code for extracting the propoer nouns encounter problem when the propoer noun contains mutiple words

matcher = Matcher(nlp.vocab)
pattern_n = [{"POS": "PROPN", "OP": "+" }] # with OP it will look for propoer noun one or more times
matcher.add("PROPER_NOUNS", [pattern_n], greedy="LONGEST")    
doc = nlp(text)
matches = matcher(doc)
matches.sort (key = lambda x: x[1])  # in case that the results will be organized in descending order based on the length (given the greedy function)
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])
    

70
(3232560085755078826, 0, 1) Australia
(3232560085755078826, 9, 10) Canadians
(3232560085755078826, 12, 13) China
(3232560085755078826, 30, 31) Canada
(3232560085755078826, 39, 44) Foreign Affairs Minister Marise Payne
(3232560085755078826, 45, 46) Sunday
(3232560085755078826, 55, 56) Australia
(3232560085755078826, 71, 72) France
(3232560085755078826, 73, 76) New York Times
(3232560085755078826, 83, 84) China


### Finding quotes

In [13]:
import os

for file in os.listdir("A1_data"): 
    print(file)

    with open ("A1_data/"+file, "r", encoding='utf-8') as f: #open all files in the folder and read them
        text = f.read()

    matcher = Matcher(nlp.vocab)
    
    pattern_q16 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '"'}] #7
    pattern_q17 = [{'ORTH': '“'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '”'}] #4
    # both curly and straight quotes
    
    matcher.add("QUOTES", [pattern_q16, pattern_q17])
    doc = nlp(text)
    matches_q = matcher(doc)
    print(len(matches_q))
    for match in matches_q[:10]:
        print(match, doc[match[1]:match[2]])
    print("\n") #blank space between outputs

5c488fac1e67d78e271d405b.txt
0


5c3436991e67d78e27d30c2c.txt
6
(16432004385153140588, 460, 480) “I walked into the store because I needed cards for school and I met this sassy woman,”
(16432004385153140588, 598, 616) “One day I changed the cake recipe and got fired for ‘creative insubordination,’”
(16432004385153140588, 761, 790) “I was introduced to someone who worked in the test kitchen of Food And Wine magazine and was asked to write a proposal for a column,”
(16432004385153140588, 991, 1006) “They were all chefs and I made them accessible for home cooks,”
(16432004385153140588, 1009, 1016) “I loved doing that.”
(16432004385153140588, 1077, 1100) “I love doing this work and I never stop thinking how lucky I am to be able to do it,”


5c339b091e67d78e27d16414.txt
1
(16432004385153140588, 634, 647) “use available resources to make educated decisions in choosing food.”


5c2afc011e67d78e27ba73a9.txt
23
(16432004385153140588, 335, 343) “Alexa, open Animal Workout.”
(1643200438515314058

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf9 in position 347: invalid start byte

In [16]:
# load a text file. Remember, you have to do 5
with open ("A1_data/5c1548a31e67d78e2771624f.txt", "r", encoding='utf-8') as f:
    text = f.read()

In [17]:
print(os.getcwd())

/Users/eunicewtk/Desktop/807comptling/LING807Han


In [18]:
# combination of different OP. 

matcher = Matcher(nlp.vocab)
pattern_q = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "!"}, {'IS_PUNCT': True, "OP": "!"}, {'ORTH': '"'}] #0
pattern_q1 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "!"}, {'IS_PUNCT': True, "OP": "?"}, {'ORTH': '"'}] #0
pattern_q2 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "!"}, {'IS_PUNCT': True, "OP": "+"}, {'ORTH': '"'}] #0
pattern_q3 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "!"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '"'}] #0

pattern_q4 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "?"}, {'IS_PUNCT': True, "OP": "!"}, {'ORTH': '"'}] #0
pattern_q5 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "?"}, {'IS_PUNCT': True, "OP": "?"}, {'ORTH': '"'}] #0
pattern_q6 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "?"}, {'IS_PUNCT': True, "OP": "+"}, {'ORTH': '"'}] #0
pattern_q7 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "?"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '"'}] #0

pattern_q8 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "!"}, {'ORTH': '"'}] #0
pattern_q9 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "?"}, {'ORTH': '"'}] #3
pattern_q10 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "+"}, {'ORTH': '"'}] #3
pattern_q11 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '"'}] #3

pattern_q12 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "*"}, {'IS_PUNCT': True, "OP": "!"}, {'ORTH': '"'}] #0
pattern_q13 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "*"}, {'IS_PUNCT': True, "OP": "?"}, {'ORTH': '"'}] #3
pattern_q14 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "*"}, {'IS_PUNCT': True, "OP": "+"}, {'ORTH': '"'}] #3
pattern_q15 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "*"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '"'}] #3

pattern_q16 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '"'}] #7
pattern_q17 = [{'ORTH': '“'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '”'}] #4

#pattern_q4 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'ORTH': '"'}]  #control
#pattern_q5 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "*"}, {'ORTH': '.,!?': True, "OP": "+"}, {'ORTH': '"'}] 
#pattern_q6 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "*"}, {'ORTH': '"'}]
#pattern_q7 = [{'ORTH': '"'}, {'IS_ALPHA': True, 'OP': '?'}, {'ORTH': '"'}]

# Load and extract from txt file 1
matcher.add("QUOTES", [pattern_q16, pattern_q17])
doc = nlp(text)
matches_q = matcher(doc)
# matches_q.sort(key = lambda x: x[1])
print (len(matches_q))
for match in matches_q[:10]:
    print (match, doc[match[1]:match[2]])

# MAYBE STRAIGHT/CURLY QUOTATION MARKS MATTER, SINCE IT IS RECOGNISING THE MIDDLE PART OF THE QUOTES AS WELL

4
(16432004385153140588, 397, 401) “road map”
(16432004385153140588, 413, 423) “should move to skills-based hiring,”
(16432004385153140588, 426, 436) “practices that prioritize … credentials and experience.”
(16432004385153140588, 628, 642) “These are things that are the least susceptible to technological disruption,”


In [20]:
# Load and extract from txt file 2
with open ("A1_data/5c489df91e67d78e271d66c5.txt", "r", encoding='utf-8') as f:
    text = f.read()
    
pattern_q16 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '"'}] #7
pattern_q17 = [{'ORTH': '“'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '”'}] #4

matcher.add("QUOTES", [pattern_q16, pattern_q17])
doc = nlp(text)
matches_q = matcher(doc)
# matches_q.sort(key = lambda x: x[1])
print (len(matches_q))
for match in matches_q[:10]:
    print (match, doc[match[1]:match[2]])

3
(16432004385153140588, 331, 351) "He enabled us to at least hang in there and get a point out of this game,"
(16432004385153140588, 358, 385) "The positive is you came in here against one of the best teams and in my opinion, we could have had that game."
(16432004385153140588, 706, 711) " Peters said. "


In [21]:
# Load and extract from txt file 3
with open ("A1_data/5c182ac21e67d78e277944ad.txt", "r", encoding='utf-8') as f:
    text = f.read()
    
pattern_q16 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '"'}] #7
pattern_q17 = [{'ORTH': '“'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '”'}] #4

matcher.add("QUOTES", [pattern_q16, pattern_q17])
doc = nlp(text)
matches_q = matcher(doc)
# matches_q.sort(key = lambda x: x[1])
print (len(matches_q))
for match in matches_q[:10]:
    print (match, doc[match[1]:match[2]])

4
(16432004385153140588, 366, 370) "very committed"
(16432004385153140588, 445, 453) "life-changing and heartbreaking,"
(16432004385153140588, 596, 630) "I would say there definitely is pressure, but I try to be as gentle and understanding with myself as I would expect to how I should treat another young person."
(16432004385153140588, 677, 694) "As young people we understand that this land is foundational to who we are,"


In [22]:
# Load and extract from txt file 4
with open ("A1_data/5c28972a795bd2fac69fa974.txt", "r", encoding='utf-8') as f:
    text = f.read()
    
pattern_q16 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '"'}] #7
pattern_q17 = [{'ORTH': '“'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '”'}] #4

matcher.add("QUOTES", [pattern_q16, pattern_q17])
doc = nlp(text)
matches_q = matcher(doc)
# matches_q.sort(key = lambda x: x[1])
print (len(matches_q))
for match in matches_q[:10]:
    print (match, doc[match[1]:match[2]])

3
(16432004385153140588, 128, 144) “My eyes are full of tears because I love this land so deeply,”
(16432004385153140588, 576, 590) “Essentially, the leader will choose the candidate in each byelection,”
(16432004385153140588, 619, 623) “whole country”


In [23]:
# Load and extract from txt file 5
with open ("A1_data/5c29beda1e67d78e27b74939.txt", "r", encoding='utf-8') as f:
    text = f.read()
    
pattern_q16 = [{'ORTH': '"'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '"'}] #7
pattern_q17 = [{'ORTH': '“'}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': '”'}] #4

matcher.add("QUOTES", [pattern_q16, pattern_q17])
doc = nlp(text)
matches_q = matcher(doc)
# matches_q.sort(key = lambda x: x[1])
print (len(matches_q))
for match in matches_q[:10]:
    print (match, doc[match[1]:match[2]])

5
(16432004385153140588, 128, 146) “The Australian Government is concerned about the recent detention of two Canadian citizens in China,”
(16432004385153140588, 265, 269) “basically kidnapping”
(16432004385153140588, 514, 518) “deep concern”
(16432004385153140588, 566, 585) “the declared motive for their investigation raises concerns about legitimate research and business practices in China.”
(16432004385153140588, 587, 623) “The denial of access to a lawyer under their status of detention is contrary to the right of defence. The EU fully supports the efforts of the Canadian Government on this matter.”


## Approach 3: Implemented version
This approach was implemented by colleagues at the [Australian Text Analytics Platform](https://www.atap.edu.au/) (ATAP). The approach is based on the [Gender Gap Tracker](https://github.com/sfu-discourse-lab/GenderGapTracker) done in the Discourse Processing Lab here at SFU. 

The first link below leads you to a binder where you can load your own files and download the output. If you prefer to do everything in your own notebook, you can download/clone the project from GitHub. 

* [Binder link](https://github.com/Australian-Text-Analytics-Platform/quotation-tool/blob/workshop_01_20220908/README.md)

    * Click on the "binder launch" button.
    * At the CILogin, under "Select an Identity Provider", go to the drop-down menu (usually default as ORCID) and select "Simon Fraser University".
    * This launches [Binder](https://mybinder.readthedocs.io/en/latest/), a service that allows you to run a notebook online on Jupyter Lab (similar to Google Colab). 
    * Run all the code cells in that notebook, uploading files from the A1_data directory. 
    * At the end, you can save the output as an Excel file. 

* [Regular GitHub project](https://github.com/Australian-Text-Analytics-Platform/quotation-tool)

    * Run the notebook "quote_extractor_notebook.ipynb"

Within the ATAP binder, upload 5 files from A1_data (the same you did for approaches 1 and 2), process them and download the results to your own computer. 

## Your turn

Check instructions on Canvas for what to do and what to submit. 

